<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Yelp-Data-Challenge---Restaurant-Recommender" data-toc-modified-id="Yelp-Data-Challenge---Restaurant-Recommender-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Yelp Data Challenge - Restaurant Recommender</a></span><ul class="toc-item"><li><span><a href="#Clean-data-and-get-rating-data" data-toc-modified-id="Clean-data-and-get-rating-data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Clean data and get rating data</a></span><ul class="toc-item"><li><span><a href="#Select-relevant-columns-in-the-original-dataframe" data-toc-modified-id="Select-relevant-columns-in-the-original-dataframe-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Select relevant columns in the original dataframe</a></span></li><li><span><a href="#Convert-rating-record-to-user-business-utility-matrix" data-toc-modified-id="Convert-rating-record-to-user-business-utility-matrix-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Convert rating record to user-business utility matrix</a></span><ul class="toc-item"><li><span><a href="#Option-1:-use-pandas.pivot_table(Dense-matrix)" data-toc-modified-id="Option-1:-use-pandas.pivot_table(Dense-matrix)-1.1.2.1"><span class="toc-item-num">1.1.2.1&nbsp;&nbsp;</span>Option 1: use pandas.pivot_table(Dense matrix)</a></span></li><li><span><a href="#Option-2:-convert-to-sparse-matrix-using-scipy.sparse.lil_matrix" data-toc-modified-id="Option-2:-convert-to-sparse-matrix-using-scipy.sparse.lil_matrix-1.1.2.2"><span class="toc-item-num">1.1.2.2&nbsp;&nbsp;</span>Option 2: convert to sparse matrix using scipy.sparse.lil_matrix</a></span></li><li><span><a href="#Create-utility-matrix-from-records" data-toc-modified-id="Create-utility-matrix-from-records-1.1.2.3"><span class="toc-item-num">1.1.2.3&nbsp;&nbsp;</span>Create utility matrix from records</a></span></li></ul></li></ul></li><li><span><a href="#Item-Item-similarity-recommender" data-toc-modified-id="Item-Item-similarity-recommender-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Item-Item similarity recommender</a></span></li><li><span><a href="#Prediction-user's-rating" data-toc-modified-id="Prediction-user's-rating-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Prediction user's rating</a></span></li><li><span><a href="#Get-the-final-recommendations-for-user" data-toc-modified-id="Get-the-final-recommendations-for-user-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Get the final recommendations for user</a></span></li><li><span><a href="#Matrix-Factorization-recommender" data-toc-modified-id="Matrix-Factorization-recommender-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Matrix Factorization recommender</a></span></li><li><span><a href="#Other-recommenders" data-toc-modified-id="Other-recommenders-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Other recommenders</a></span></li></ul></li></ul></div>

# Yelp Data Challenge - Restaurant Recommender

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
% matplotlib inline
plt.style.use("ggplot")

In [63]:
df = pd.read_csv('last_year_restaurant_reviews.csv')

In [82]:
df.columns.values
df.head()
df["business_id"]


0          --6MefnULPED_I942VcFNA
1          --6MefnULPED_I942VcFNA
2          --6MefnULPED_I942VcFNA
3          --6MefnULPED_I942VcFNA
4          --6MefnULPED_I942VcFNA
5          --6MefnULPED_I942VcFNA
6          --6MefnULPED_I942VcFNA
7          --6MefnULPED_I942VcFNA
8          --6MefnULPED_I942VcFNA
9          --6MefnULPED_I942VcFNA
10         --6MefnULPED_I942VcFNA
11         --6MefnULPED_I942VcFNA
12         --7zmmkVg-IMGaXbuVd0SQ
13         --7zmmkVg-IMGaXbuVd0SQ
14         --7zmmkVg-IMGaXbuVd0SQ
15         --7zmmkVg-IMGaXbuVd0SQ
16         --7zmmkVg-IMGaXbuVd0SQ
17         --7zmmkVg-IMGaXbuVd0SQ
18         --7zmmkVg-IMGaXbuVd0SQ
19         --7zmmkVg-IMGaXbuVd0SQ
20         --7zmmkVg-IMGaXbuVd0SQ
21         --7zmmkVg-IMGaXbuVd0SQ
22         --7zmmkVg-IMGaXbuVd0SQ
23         --7zmmkVg-IMGaXbuVd0SQ
24         --7zmmkVg-IMGaXbuVd0SQ
25         --7zmmkVg-IMGaXbuVd0SQ
26         --7zmmkVg-IMGaXbuVd0SQ
27         --7zmmkVg-IMGaXbuVd0SQ
28         --7zmmkVg-IMGaXbuVd0SQ
29         --8

In [65]:
df['date'].value_counts()

2017-08-12    4614
2017-08-13    4340
2017-07-14    4269
2017-07-16    4185
2017-07-08    4159
2017-07-30    4038
2017-07-09    4025
2017-07-15    3998
2017-07-29    3973
2017-10-09    3949
2017-06-25    3938
2017-08-06    3922
2017-06-09    3912
2017-10-08    3893
2017-07-22    3845
2017-07-23    3838
2017-08-05    3832
2017-02-19    3831
2017-09-17    3826
2017-11-07    3816
2017-09-06    3804
2017-04-30    3796
2017-08-19    3783
2017-03-26    3766
2017-08-20    3759
2017-09-03    3756
2017-06-11    3753
2017-06-26    3751
2017-07-17    3742
2017-07-01    3741
              ... 
2017-03-02    2832
2017-10-06    2820
2017-03-01    2808
2017-10-02    2807
2017-03-09    2805
2017-02-09    2804
2017-10-03    2800
2017-01-27    2766
2017-01-18    2764
2017-02-10    2746
2017-01-13    2744
2017-02-03    2723
2017-01-25    2704
2017-02-02    2649
2017-02-07    2627
2017-01-19    2622
2017-01-26    2622
2017-01-10    2598
2017-01-24    2587
2017-10-31    2573
2017-01-12    2525
2017-02-06  

In [66]:
subset_condition = df['date']>'2017-12-01'
subset_df = df[subset_condition]

##  Clean data and get rating data 

### Select relevant columns in the original dataframe

In [67]:
# Get business_id, user_id, stars for recommender
selected_df = subset_df[['business_id', 'user_id','stars_review']]
selected_df.head()
selected_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31345 entries, 8 to 1126054
Data columns (total 3 columns):
business_id     31345 non-null object
user_id         31345 non-null object
stars_review    31345 non-null float64
dtypes: float64(1), object(2)
memory usage: 979.5+ KB


In [68]:
selected_df = selected_df.dropna(how='any')
selected_df
selected_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31345 entries, 8 to 1126054
Data columns (total 3 columns):
business_id     31345 non-null object
user_id         31345 non-null object
stars_review    31345 non-null float64
dtypes: float64(1), object(2)
memory usage: 979.5+ KB


In [70]:
# create new number indexes for business and users
b_i = 0
u_i = 0
business_id_trans = {}
business_id_n = []
user_id_trans ={}
user_id_n =[]

reverse_business_id_trans = {}
reverse_user_id_trans = {}

for _, row in selected_df.iterrows():
    if row.business_id in business_id_trans.keys():
        business_id_n.append(business_id_trans[row.business_id])
    else:
        business_id_trans[row.business_id]=b_i
        reverse_business_id_trans[b_i]=row.business_id
        business_id_n.append(business_id_trans[row.business_id])
        b_i = b_i+1
    if row.user_id in user_id_trans.keys():
        user_id_n.append(user_id_trans[row.user_id])
    else:
        user_id_trans[row.user_id]=u_i
        reverse_user_id_trans[u_i]=row.user_id
        user_id_n.append(user_id_trans[row.user_id])
        u_i = u_i+1

### Create reverse index to get the restaurant information for evaluation 

In [90]:
print "reverse_business_id_trans: ", reverse_business_id_trans[16592]
print "reverse_user_id_trans: ", reverse_user_id_trans[20355]

# This command below helps get the content regarding certain business_id
print df.loc[df['business_id'] == "rnvsL0oFZpzpO61GXqBF6g"]["attributes"].head()

reverse_business_id_trans:  rnvsL0oFZpzpO61GXqBF6g
reverse_user_id_trans:  F7HO9icnRADHLlwsoBzaSA
978859    {u'GoodForMeal': {u'dessert': False, u'latenig...
978860    {u'GoodForMeal': {u'dessert': False, u'latenig...
978861    {u'GoodForMeal': {u'dessert': False, u'latenig...
978862    {u'GoodForMeal': {u'dessert': False, u'latenig...
978863    {u'GoodForMeal': {u'dessert': False, u'latenig...
Name: attributes, dtype: object


In [20]:
# Dictionary of id transformation for business
business_id_trans

{'rnvsL0oFZpzpO61GXqBF6g': 16592,
 'XYIPXJ9parr9FtvvcGI1SA': 10163,
 'e_hNMR1z_jfvSJ1TqG90MQ': 12637,
 'ZEMiqiTvsI1_SrVRHsGN1w': 10680,
 'WvJNkHE_hcg8Hu8Ffz_Klw': 9977,
 'LlnHZHEupJtr0YdIONqGGA': 6701,
 '0cf-w9PK5gLkBkitL3bdqw': 514,
 'jDZ5VPEIeh7oJcenCpJcXg': 14064,
 'YzlgswAwmpkMQifS3-YQTw': 10603,
 'Vlp68XpPYH0G9NHkgcglvA': 9629,
 'Xxs_kfoDirWGR4nD5Spfbw': 10299,
 '2j78b8m1w4Sr2GVGCzFBtQ': 1088,
 'wX5LyYT2Tb-EdCr0gyLymw': 18044,
 'inRcEs2AbhpuBzipwP1x8g': 13924,
 'EBHzEtuOJz474NwRQFqJbg': 4436,
 'y6R_bSCYiotoUjvygVcFaQ': 18537,
 'p8AwMheeFJ_8pP2EpOAPsA': 15795,
 'eETFuqEvIpCpkNchkFXO6Q': 12529,
 '2SjbysAT-GKznUIxuJoKtg': 1011,
 'uYtlrzBwtekPt8GKwUBoWw': 17429,
 'Q-dVQgyof9paXKJLwFsn3Q': 7893,
 '2Be0jvQQvBcmjqdezM4Hjg': 926,
 'Qvjy0HpGCLM7XBZjNeMc3w': 8164,
 'bVqNtwcwIz2gAacHWYF2lw': 11687,
 'gTedBMNAi5pBy0amenPOYQ': 13200,
 '_y1Et7f4NE6D1P1GzphlIw': 11221,
 '6uORuTgTKIUvfA34WoGgWw': 2314,
 '4hWDMVtfnpyY72_5QMbthA': 1676,
 'sEZU0ssY_7GlGKeBntkX-A': 16724,
 'NfXIYXL42neKagrXMtcCZg': 7

In [15]:
# Dictionary of id transformation for user
user_id_trans

{'F7HO9icnRADHLlwsoBzaSA': 20355,
 'UmxHEMUR1jPhICpiU1prvA': 13510,
 '5MmAhdLK63hc1iAh8aTC1Q': 15701,
 'vK-2CNiCaiPb3jcf-2oQwA': 8322,
 '4a6eAFBAIC3ZuNIF8gmxow': 18762,
 'MLS-fb5nCooq0aYz1pVlWg': 11324,
 'XsKL7KGNXL1r_YTxXuUWkA': 4472,
 'SAdmlwTOxA9p-1GLamEssw': 13624,
 'M6_4Mx7cVaEa8SbBV8gtTg': 1969,
 'KhRu7TyqHFr15kf2TNvTEw': 9893,
 'OrkFCDG7PxLVXNvzkk3P7Q': 11221,
 'wlZFDe22ELMJJc9O0w62TA': 16109,
 'J0MaPU4yjb-CJapxyVoL8g': 5790,
 '-bFSPx2tvhxlw46q5RdFAA': 9256,
 'pTLgvwLwek6JehIOtjyNgA': 7778,
 'jBhrfqqnDJ-VoTu5DWeMYQ': 7031,
 'Nag61x7xZMRvVtXDw9ADjQ': 12736,
 'bzAmq-ufXNAcomk-47Bu7w': 13055,
 'Cy80vakyzvbZfrsbldOElw': 4375,
 'Alg7McvJqz8obZQ57YGzRQ': 13995,
 '_fZO39KOL1aOjFAhza5aDA': 20864,
 'Y3E2-eNu9_QibkLeEfHrMA': 5771,
 '4CDvfJH9L3TKeuQCjfBZkg': 592,
 'uZOvIc6r_rqlSTaulIYgJQ': 12794,
 'MnrrAzIfPaYNFuHU1igKBw': 321,
 'D_JXcH4VKHQpKIuJgsw0hQ': 17195,
 'CW1XEtbKm60ucBqdyylnkw': 3995,
 'PNo1yZIVIWm05XPcjDoKlA': 19424,
 'wDaOs0bYmt38fBf4J2f8EA': 16662,
 'LqTDuRW5CJ5vcd0e4RVMcA': 14

In [16]:
# index for business
business_id_n

[0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 3,
 3,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 16,
 16,
 17,
 18,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 26,
 27,
 28,
 29,
 30,
 31,
 31,
 31,
 31,
 32,
 33,
 33,
 34,
 34,
 34,
 35,
 35,
 35,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 41,
 42,
 43,
 44,
 45,
 46,
 46,
 47,
 48,
 49,
 50,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 52,
 52,
 52,
 53,
 54,
 55,
 56,
 56,
 56,
 57,
 58,
 59,
 59,
 59,
 59,
 59,
 59,
 60,
 61,
 62,
 62,
 62,
 62,
 62,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 69,
 69,
 69,
 70,
 71,
 72,
 72,
 72,
 72,
 73,
 74,
 75,
 76,
 76,
 76,
 77,
 78,
 79,
 79,
 79,
 80,
 81,
 82,
 82,
 83,
 83,
 84,
 85,
 86,
 87,
 88,
 88,
 88,
 89,
 90,
 91,
 91,
 92,
 93,
 94,
 94,
 95,
 95,
 95,
 95,
 95,
 95,
 96,
 97,
 98,
 99,
 100,
 100,
 100,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112

In [17]:
# index for user
user_id_n

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 3,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 102,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 1

In [19]:
selected_df['business_id_n']=business_id_n
selected_df['user_id_n']=user_id_n
selected_df = selected_df[['business_id_n', 'user_id_n','stars_review']]
selected_df = selected_df.reset_index(drop=True)
selected_df.describe()
print selected_df

       business_id_n  user_id_n  stars_review
0                  0          0           1.0
1                  1          1           5.0
2                  1          2           5.0
3                  1          3           5.0
4                  1          4           1.0
5                  1          5           5.0
6                  1          6           5.0
7                  1          7           3.0
8                  1          8           5.0
9                  1          9           4.0
10                 1         10           5.0
11                 1         11           4.0
12                 1         12           5.0
13                 1         13           5.0
14                 2         14           5.0
15                 3         15           5.0
16                 3         16           4.0
17                 3         17           2.0
18                 4         18           5.0
19                 5         19           1.0
20                 6         20   

### Convert rating record to user-business utility matrix

####  Option 1: use pandas.pivot_table(Dense matrix)

In [26]:
# #There are many users that haven't given many reviews, exclude these users from the item-item similarity recommender
# users_rated_condition = df['stars_review']!=''
# pure_df = df[users_rated_condition]
# pure_df.info()

In [27]:
### TZ:not sure why we need to use pivot table

In [28]:
df_utility = pd.pivot_table(data=selected_df[:10000], values='stars_review', 
                            index='user_id_n',columns='business_id_n',fill_value=0) 
# we don't really want null values become 0, cuz we would do predict on these users

In [29]:
df_utility.head()

business_id_n,0,1,2,3,4,5,6,7,8,9,...,6075,6076,6077,6078,6079,6080,6081,6082,6083,6084
user_id_n,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
df_utility.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8611 entries, 0 to 8610
Columns: 6085 entries, 0 to 6084
dtypes: int64(6085)
memory usage: 399.8 MB


####  Option 2: convert to sparse matrix using scipy.sparse.lil_matrix

In [31]:
### TZ:not sure why we need to convert, faster??

In [35]:
n_columns = selected_df['user_id_n'].max() 
n_rows = selected_df['business_id_n'].max()
rating_mat = sparse.lil_matrix((n_columns,n_rows))
print (n_columns, n_rows)
print "rating_mat:", rating_mat
### TZ: so sparse matrix is empty the above is just initializing

(22726, 19104)
rating_mat: 


#### Create utility matrix from records

In [37]:
for _, row in selected_df.iterrows():
    rating_mat[row.user_id_n-1, row.business_id_n-1] = row.stars_review
# substract 1 from id due to index starting from 0    

In [46]:
print rating_mat.shape
print "Rating matrix: \n", rating_mat


(22726, 19104)
Rating matrix: 
  (0, 0)	5.0
  (1, 0)	5.0
  (1, 1561)	3.0
  (1, 1858)	5.0
  (1, 5612)	5.0
  (2, 0)	5.0
  (2, 14)	2.0
  (3, 0)	1.0
  (3, 13562)	5.0
  (3, 14187)	5.0
  (3, 14279)	3.0
  (4, 0)	5.0
  (4, 3921)	2.0
  (4, 5895)	5.0
  (4, 14846)	4.0
  (5, 0)	5.0
  (6, 0)	3.0
  (7, 0)	5.0
  (7, 760)	3.0
  (7, 4568)	4.0
  (8, 0)	4.0
  (8, 12947)	4.0
  (8, 13575)	4.0
  (9, 0)	5.0
  (10, 0)	4.0
  (11, 0)	5.0
  (11, 3076)	5.0
  (11, 7981)	5.0
  (11, 8373)	5.0
  (11, 8526)	5.0
  (12, 0)	5.0
  (13, 1)	5.0
  (13, 10058)	5.0
  (14, 2)	5.0
  (15, 2)	4.0
  (15, 1007)	4.0
  (15, 1087)	4.0
  (15, 1140)	4.0
  (15, 1959)	4.0
  (15, 2003)	5.0
  (15, 3504)	4.0
  (15, 3591)	4.0
  (15, 5641)	5.0
  (15, 6204)	5.0
  (15, 6624)	4.0
  (15, 7123)	5.0
  (15, 7950)	5.0
  (15, 9152)	3.0
  (15, 9293)	4.0
  (15, 9941)	4.0
  (15, 10857)	4.0
  (15, 10952)	4.0
  (15, 12104)	5.0
  (15, 14398)	4.0
  (15, 14739)	5.0
  (15, 16273)	2.0
  (15, 16530)	4.0
  (15, 16920)	3.0
  (16, 2)	2.0
  (16, 15215)	5.0
  (17, 3)	5

## Item-Item similarity recommender

In [47]:
### TZ: Oh, this is why, we need rating_mat so we can calculate cosine_similarity!! 
### TZ: the same item will get 1 as the value of cosine similarity
### TZ: But we only use user_id and business_id ??? that's not enough though..

item_sim_mat = cosine_similarity(rating_mat)

In [48]:
print(item_sim_mat)
print item_sim_mat.shape

[[ 1.          0.54554473  0.92847669 ...,  0.          0.          0.        ]
 [ 0.54554473  1.          0.50652556 ...,  0.          0.          0.        ]
 [ 0.92847669  0.50652556  1.         ...,  0.          0.          0.        ]
 ..., 
 [ 0.          0.          0.         ...,  1.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          1.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          1.        ]]
(22726, 22726)


In [49]:
least_to_most_sim_indexes = np.argsort(item_sim_mat,axis=1)

In [50]:
least_to_most_sim_indexes2 = np.argsort(item_sim_mat[10000:20000],axis=1)

In [51]:
least_to_most_sim_indexes3 = np.argsort(item_sim_mat[20000:],axis=1)

In [52]:
neighborhood_size = 100
neighborhoods = least_to_most_sim_indexes[:,-neighborhood_size:]
neighborhoods

array([[ 7551,  7614,  7613, ...,    12,     5,     0],
       [ 7625,  7607,  7808, ...,    12,     0,     1],
       [ 7611,  7610,  7609, ...,     5,     0,     2],
       ..., 
       [ 7486,  7485,  7484, ...,  7577, 22725, 22723],
       [ 7488,  7487,  7486, ...,  7569,  7545, 22724],
       [ 7488,  7487,  7486, ...,  7569,  7545, 22725]])

## Prediction user's rating

In [53]:
user_id = 911

In [94]:
n_users = df_utility.shape[0]
n_items = rating_mat.shape[1]
# To find the items rated according to the id of users
items_reted_by_this_user = rating_mat[user_id].nonzero()[1]

# create a empty list for storing each item's rating a user might give to
out = np.zeros(n_items)

# Iterate to calculate the rating using item-based formulation
# TZ: so basically go over all the item and find similar items to the ones that's being rated by this user \
# and put them in an array


for item_to_rate in range(n_items):
#     print "item_to_rate: ", item_to_rate
    # Find the overlap item id both in neighborhood set and items rated by the specific user that we want to predict
    relevant_items = np.intersect1d(neighborhoods[item_to_rate],items_reted_by_this_user)
    # write result into 'out' list by using item-based formulation
    out[item_to_rate] = rating_mat[user_id,relevant_items]*item_sim_mat[item_to_rate,relevant_items]/item_sim_mat[item_to_rate,relevant_items].sum()

# print out
    
pre_ratings =  np.nan_to_num(out)
# print pre_ratings
for i in range(len(pre_ratings)):
    if pre_ratings[i]!=0:
        print "index: i: ", i, " rating: ", (pre_ratings[i])
        
        
#### TZ: TODO: Item ID is missing for the rating prediction 

index: i:  8  rating:  4.0
index: i:  207  rating:  4.0
index: i:  554  rating:  4.0
index: i:  555  rating:  4.0
index: i:  556  rating:  4.0
index: i:  557  rating:  4.0
index: i:  558  rating:  4.0
index: i:  1295  rating:  3.0
index: i:  2034  rating:  4.0
index: i:  2206  rating:  4.0
index: i:  2449  rating:  4.0
index: i:  2465  rating:  4.0
index: i:  4485  rating:  4.0
index: i:  4486  rating:  4.0
index: i:  5374  rating:  5.0
index: i:  5375  rating:  5.0
index: i:  5376  rating:  5.0
index: i:  6361  rating:  5.0
index: i:  6704  rating:  4.0
index: i:  7113  rating:  3.0
index: i:  7860  rating:  3.0
index: i:  8376  rating:  4.0
index: i:  9478  rating:  5.0
index: i:  9479  rating:  5.0
index: i:  12437  rating:  3.0
index: i:  12504  rating:  3.0
index: i:  13012  rating:  4.0
index: i:  13059  rating:  4.0
index: i:  14136  rating:  4.0
index: i:  14257  rating:  4.0
index: i:  14799  rating:  3.0
index: i:  14890  rating:  3.0
index: i:  14929  rating:  4.0
index: i: 

## Get the final recommendations for user

In [ ]:
# the number of recommendation for restaurants
recommend_n =20

# Get item indexes sorted by prediected rating
item_index_sorted_by_pre_rating = list(np.argsort(pre_ratings))[::-1]

# Find items that have been rated by user
items_reted_by_this_user = rating_mat[user_id].nonzero()[1]

# exclude the item that has been rated by user
unrated_items_by_pre_rating = [item for item in item_index_sorted_by_pre_rating if item not in items_reted_by_this_user]

unrated_items_by_pre_rating[:recommend_n]

[5374,
 9478,
 9479,
 5375,
 2034,
 2449,
 4486,
 16543,
 14257,
 16542,
 13059,
 16544,
 16545,
 8376,
 16531,
 16530,
 16547,
 16532,
 16533,
 16534]

In [ ]:
## Matrix Factorization recommender

In [ ]:
#  Compare two of the methods demoed in Practice Class: sklearn NMF, sklearn TruncatedSVD, or GraphLab

In [ ]:
## Other recommenders 
# * Other features (have you noticed there are other features in the Yelp dataset, e.g. tips, etc.?)
# * Popularity-based
# * Content-based
# * Hybrid

### Recommendation evaluation: 